In [1]:
# import packages
import numpy as np
import cadquery as cq
from scipy.stats import qmc
import math
import itertools
import matplotlib.pyplot as plt
import os
import json
import copy
import random

# Load inputs

In [2]:
# Read the JSON file as a dictionary
# NOTE: when reading in from JSON, the key is read as a string
json_filepath = '/glade/u/home/joko/spherical-code/data/s_code.json'
with open(json_filepath, 'r') as json_file:
    s_code_dict = json.load(json_file)

# Source

# Main